## Keras Layered Model & Explorations - JoanieW

Based on: https://elix-tech.github.io/ja/2016/06/02/kaggle-facial-keypoints-ja.html#conv

And modeled after Jackie's code structure for Lenet5

In [1]:
#Set the utils path to point to the utils directory locally
UTILS_PATH = "update with full path to utils director"

import os, sys
sys.path.append(UTILS_PATH)
from load_models import LoadTrainModels
from predict_models import PredictModels
from transform_data import TransformData
import imp
import pickle
import numpy as np
import pandas as pd
from keras.models import Sequential, Model, model_from_json

In [ ]:
cd ..

In [2]:
### this is gold right here.
file_path = "cleantrain/"
trainer = LoadTrainModels("data/models/", file_path)
trainer.print_paths()

Model dir: C:/data/Jackie_Lenet5/


### For every version of a clean Train file in a given path, create and save a model

In [3]:

files = os.listdir(file_path)

#For every version of a cleaned Train file in CleanTrain directory, create and save a model
for filename in files: 
    print("Opening file: ", filename)
    clean_file = "".join((file_path,filename))
    print(clean_file)
    train_data = pickle.load( open( clean_file, "rb" ) )
    train_data = train_data.drop(['level_0', 'check_sum', 'index'], axis=1,errors='ignore')
    print("Train Shape:", train_data.shape)

    filename = str(filename).replace('.p', '').strip()
    print("Begin model and train:")
    model_name = "".join((filename,"_jcw"))
    print("Model name:", model_name)
    #Run with separate = True to train on 8 columns of data
    model, history = trainer.train_jcw(model_name, train_data,verbose = True, separate = True)
    print("End model and train")    
    print()

Opening file:  clean_all_outliers.p
C:/Data/CleanTrain_1/clean_all_outliers.p
Train Shape: (7041, 31)
Begin model and train:
Model name: clean_all_outliers_jcw
Scaling images
Scaling complete.
Index(['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x',
       'right_eye_center_y', 'nose_tip_x', 'nose_tip_y',
       'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y', 'image'],
      dtype='object')
Begining the split of Train with all features
Looking for model JW
JW model file not found. Model creation beginning
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        288       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)  

ValueError: in user code:

    C:\Users\mspuc\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\mspuc\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\mspuc\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\mspuc\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\mspuc\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\mspuc\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
 

### For every model file in a given path, predict using the model and save the predictions in CSV file

In [ ]:


id_lookup = pickle.load( open( "data/id_lookup.p", "rb" ) )
test = pickle.load( open( "data/test.p", "rb" ) )

#Using local paths as this is way faster...
file_path = "data/models/"
pred_path = "data/predictions/"

predictor = PredictModels(file_path,pred_path , id_lookup)

predictor.print_paths()

files = os.listdir(file_path)
#For every model in file_path, predict using the model and save the predictions in CSV file
for filename in files:
    if ".h5" in filename:
        base_name = filename[:-3]
        model_json = ''.join((base_name,".json"))
        print("Working with: ", base_name)
        print("Begin Predict")
        #Set columns = False if only predicting 8 keypoints
        Y= predictor.predict_standard(base_name, filename, model_json, test, columns=False)
        print("End model and train")    
        print()


## Explorations on using different models for 8 vs. 30 predictions

In [ ]:


id_lookup = pickle.load( open( "data/id_lookup.p", "rb" ) )
test = pickle.load( open( "data/test.p", "rb" ) )

In [ ]:
#Filtering to ids where image_id count == 8
new_id = id_lookup.groupby("image_id").agg('count')
new_id_2 = new_id[new_id[ "row_id" ] <= 8]
new_id_1 = new_id[new_id[ "row_id" ] > 8]
print(new_id_2.index)
print(new_id_1.index)


In [ ]:
id_lookup_1 = id_lookup[id_lookup['image_id']<=591]
id_lookup_2 = id_lookup[id_lookup['image_id'] > 591]

In [ ]:
test_1 = test[test['image_id']<=591]
test_1

In [ ]:
test_2 = test[test['image_id']>591]
test_2

In [ ]:
pickle.dump(test_2,open("data/test_8keypoints.p", "wb"))

In [ ]:
#Identify where in the id_lookup is the last output row
print(np.max(id_lookup_1['row_id']))
print(np.min(id_lookup_1['row_id']))

max_row_full = np.max(id_lookup_1['row_id'])

The first 591 rows of the id lookups require more than 8 keypoints. Afterwards the next 1192 rows only require 6 or 8 keypoints.

In [ ]:
id_lookup = id_lookup_2
test = test_2
pred_path = "data/predictions/"

predictor = PredictModels(file_path,pred_path , id_lookup)

model_json = "data/models/clean_w_outliers_jcw.json"
model_file = "data/models/clean_w_outliers_jcw.h5"
json_file = open(model_json, "r")
model_json_data = json_file.read()
json_file.close()
model = model_from_json(model_json_data)
model.load_weights(model_file)
base_name = "clean_w_outliers_jcw"
filename = model_file
Y= predictor.predict_standard(base_name, filename, model_json, test)

In [ ]:
cd intro_ml/blackboxes/

## Reading in prediction csvs to combine to make final prediction file

In [ ]:
file_path = "data/predictions/"
files = os.listdir(file_path)

for file in files:
    #separate is from the most recent run of predictions based solely on train data that had 8 keypoints
    predictions_separate= pd.read_csv("data/predictions/"+file)
    #full is the older predictions data that was built on all the train
    predictions_full = pd.read_csv("data/predictions_full/"+file)
    new_pred = predictions_separate[predictions_separate['RowId']>max_row_full].copy()
    new_pred = new_pred.append(predictions_full[predictions_full['RowId']<=max_row_full].copy())
    new_pred = new_pred.sort_values(by=['RowId'])
    new_pred.to_csv("combined_"+file,index=False)
    